In [1]:
!pip install pyarrow pylance lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 45.6 MB/s eta 0:00:00


In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import lightning
import lance
import pyarrow as pa

from tqdm.auto import tqdm
from transformers import GPT2TokenizerFast
from datasets import load_dataset

print("Imported all libraries!")

Imported all libraries!


In [3]:
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
dataset = load_dataset("roneneldan/TinyStories", data_files={"train": "TinyStoriesV2-GPT4-train.txt"})

README.md:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

TinyStoriesV2-GPT4-train.txt:   0%|          | 0.00/2.23G [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [5]:
all_tokens = []
total_rows = 1000

data = dataset["train"].select([x for x in range(total_rows)])

a = 0
l = 0
for row in tqdm(data["text"]):
    text = row.split(" ")
    print(text)
    l += len(text)
    a += 1
    if a>=3:
        break
print(l)

  0%|          | 0/1000 [00:00<?, ?it/s]

['']
['Once', 'upon', 'a', 'time', 'there', 'was', 'a', 'little', 'boy', 'named', 'Ben.', 'Ben', 'loved', 'to', 'explore', 'the', 'world', 'around', 'him.', 'He', 'saw', 'many', 'amazing', 'things,', 'like', 'beautiful', 'vases', 'that', 'were', 'on', 'display', 'in', 'a', 'store.', 'One', 'day,', 'Ben', 'was', 'walking', 'through', 'the', 'store', 'when', 'he', 'came', 'across', 'a', 'very', 'special', 'vase.', 'When', 'Ben', 'saw', 'it', 'he', 'was', 'amazed!', '', '']
['He', 'said,', '“Wow,', 'that', 'is', 'a', 'really', 'amazing', 'vase!', 'Can', 'I', 'buy', 'it?”', '']
74


In [6]:
i = 0
for row in tqdm(data["text"], total=len(data)):
    row = row.replace("<|endoftext|>", " ")
    encoded = tokenizer(row)["input_ids"]
    all_tokens.extend(encoded)


pa_table = pa.Table.from_arrays([all_tokens], names=["value"])
lance.write_dataset(pa_table, "tiny_stories_gpt4_encoded.lance", {"model": "create"})
print(f"Total tokens in the tokenized dataset: {len(all_tokens)}")

  0%|          | 0/1000 [00:00<?, ?it/s]

Total tokens in the tokenized dataset: 31603


In [7]:
class Config:
    vocab_size = 50304
    n_epochs = 30
    batch_size = 36
    lr = 3e-4
    wd = 1e-6
    n_embed = 256
    num_blocks = 12
    num_heads = 12
    head_size = n_embed//num_heads
    context_len = 224
    attn_dropout_val = 0.2
    mha_dropout_val = 0.2
    ffn_dropout_val = 0.2

In [8]:
class CasualAttentionHead(nn.Module):
    def __init__(self, config):
        super(CasualAttentionHead, self).__init__()
        self.config = config

        self.query = nn.Linear(config.n_embed, config.head_size, bias=False)
        self.key = nn.Linear(config.n_embed, config.head_size, bias=False)
        self.value = nn.Linear(config.n_embed, config.head_size, bias=False)
        self.attn_drop = nn.Dropout(config.attn_dropout_val)
        # mask for casual attention during training
        self.register_buffer("mask", torch.tril(torch.ones(config.context_len, config.context_len)))

    def forward(self, x):
        bs, context_len, embed_dim = x.shape
        q, k, v = self.query(x), self.key(x), self.value(x)
        attn_filter = torch.divide(torch.bmm(q, k.transpose(1, 2)), self.config.head_size)
        attn_filter = attn_filter.masked_fill(self.mask[:context_len, :context_len]==0, float("-inf"))
        attn_weights = F.softmax(attn_filter, dim=-1)
        attn_weights = self.attn_drop(attn_weights)
        output = torch.bmm(attn_weights, v)
        return output

class MultiHeadedAttention(nn.Module):
    def __init__(self, config):
        super(MultiHeadedAttention, self).__init__()
        self.config = config
        self.heads = nn.ModuleList(
            [CasualAttentionHead(config) for _ in range(config.num_heads)]
        )
        self.proj = nn.Linear(config.num_heads*config.head_size, config.n_embed)
        self.mha_drop = nn.Dropout(config.mha_dropout_val)

    def forward(self, x):
        mha_output = torch.cat([head(x) for head in self.heads], dim=-1)
        return self.mha_drop(self.proj(mha_output))

class FeedForwardNetwork(nn.Module):
    def __init__(self, config):
        super(FeedForwardNetwork, self).__init__()

        self.ffn = nn.Sequential(
            nn.Linear(config.n_embed, config.n_embed*4),
            nn.GELU(),
            nn.Linear(config.n_embed*4, config.n_embed),
            nn.Dropout()
        )
    def forward(self, x):
        return self.ffn(x)

In [9]:
class Block(nn.Module):
    def __init__(self, config):
        super(Block, self).__init__()
        self.mha = MultiHeadedAttention(config)
        self.ln1 = nn.LayerNorm(config.n_embed)
        self.ffn = FeedForwardNetwork(config)
        self.ln2 = nn.LayerNorm(config.n_embed)

    def forward(self, x):
        x = self.ln1(x+self.mha(x))
        x = self.ln2(x+self.ffn(x))
        return x

In [10]:
class GPT(lightning.LightningModule):
    def __init__(self, config):
        super(GPT, self).__init__()
        self.config = config
        self.save_hyperparameters()
        self.token_embedding = nn.Embedding(config.vocab_size, config.n_embed)
        self.positional_embedding = nn.Embedding(config.context_len, config.n_embed)
        self.backbone = nn.Sequential(*[Block(config) for _ in range(config.num_blocks)])
        self.lm_head = nn.Linear(config.n_embed, config.vocab_size)

    def forward(self, x):
        tok_emb = self.token_embedding(x)
        pos_emb = self.positional_embedding(torch.arange(x.shape[1], device=self.device))
        x = tok_emb+pos_emb
        x = self.backbone(x)
        logits = self.lm_head(x)
        return logits

    def get_loss(self, predictions, target):
        B, C, V = predictions.shape
        predictions = predictions.view(B*C, V)
        target = target.view(B*C)
        loss = F.cross_entropy(predictions, target)
        return loss

    def training_step(self, batch, batch_idx):
        text, target = batch
        text = text.long()
        target = target.long()
        logits = self(text)
        loss = self.get_loss(logits, target)

        self.log('loss', loss.item(), prog_bar=True)
        logs = {'loss': loss}

        return {"log": logs, "loss": loss}

    def training_end(self, outputs):
        avg_loss = torch.stack([x['log']['loss'] for x in outputs]).mean()
        logs = {"log": avg_loss}
        print(f"val_loss: {avg_loss}")
        return {"log": logs}

    def configure_optimizers(self):
        opt = torch.optim.AdamW(self.parameters(), lr=self.config.lr, weight_decay=self.config.wd)
        return [opt], []

In [11]:
def generate(model, prompt, max_tokens, temperature=0.7):
    model.eval()
    for _ in range(max_tokens):
        prompt = prompt[:, :config.context_len]
        logits = model(prompt)
        logits = logits[:, -1, :] / temperature
        logit_probs = nn.functional.softmax(logits, dim=-1)
        next_prompt = torch.multinomial(logit_probs, num_samples=1)
        prompt = torch.cat((prompt, next_prompt), dim=1)
    return prompt

In [12]:
class GPTDataset(Dataset):
    def __init__(self, dataset_path, context_len):
        self.ds = lance.dataset(dataset_path)
        self.context_len = context_len
        self.length = self.ds.count_rows()-context_len

    def __len__(self):
        return self.length

    def from_idxs(self, idxs):
        data = self.ds.take(idxs).to_pylist()
        data = torch.tensor(list(map(lambda x: x["value"], data)))
        return data

    def __getitem__(self, idx):
        current_window_idxs = np.arange(idx, idx+self.context_len+1)
        data = self.from_idxs(current_window_idxs)
        x = data[0:self.context_len]
        y = data[1:self.context_len+1]
        return x, y

In [13]:
if __name__ == "__main__":
    dataset_path = "tiny_stories_gpt4_encoded.lance"
    config = Config()
    gpt = GPT(config)
    dataset = GPTDataset(dataset_path, config.context_len)
    loader = DataLoader(
        dataset,
        batch_size=config.batch_size,
        shuffle=False
    )
    trainer = lightning.Trainer(default_root_dir="/kaggle/working/", accelerator="auto", max_epochs=config.n_epochs)
    trainer.fit(gpt, loader)

INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name                 | Type       | Params | Mode 
------------------------------------------------------------
0 | token_embedding      | Embedding  | 12.9 M | train
1 | positional_embedding | Embedding  | 57.3 K | train
2 | backbone             | Sequential | 9.4 M  | train
3 | lm_head              | Linear     | 12.9 M | train
------------------------------------------------------------
35.3 M    Trainable params
0         Non-trainable params
35.3 M    Total params
141.128   Total estimated model params size (MB)
880       Modules in train mode
0         Modules in eval mode
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_wo

Training: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=30` reached.


In [14]:
prompt = "My dog is"
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
gpt = gpt.to('cuda')
prompt = tokenizer.encode(prompt, return_tensors='pt').to('cuda')
generated_text = generate(gpt, prompt, max_tokens=config.context_len, temperature=0.7)
generated_text = tokenizer.decode(generated_text.tolist()[0])
print(generated_text)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


My dog is. to She.. and.. the the day a..,... on,.. the. Tim to on... bird it saw want. the his. a and the aOh the, with the Bob and. into He day a the. the not, The to..,. a the... and. and.,. floors,.. to of ",. the, the and the.  the and her, a big He,. the.. little a,, the was was was,.. the.,,. Tom the!. to the. and,., a., the, to., made. the all it Tom.,. was They the and,,. it dinner,, to... big., it., I a. Tim help told the. take, Bob,,  him big and.. the the He a scared and was.. to  bird." was was the not, to... the.,.,, mom., the found.
